## Подключение диска и установка keras.tunes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.5 MB/s eta 0:00:00


# Инициализация

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Dropout, LSTM, Bidirectional, SpatialDropout1D, Flatten, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from kerastuner.tuners import BayesianOptimization
from tensorflow import keras
from keras_tuner import Objective
import pandas as pd

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


<ipython-input-3-8e3e93baa2c2>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization


In [ ]:
# Максимальное количество слов
num_words = 10000
# Максимальная длина комментария
max_comment_len = 50
#Число классов
num_classes = 6

# Загрузка тренировочных данных

In [ ]:
train = pd.read_csv('drive/MyDrive/toxicCommentsDataSet/toxicCommentsTrain.csv')

In [ ]:
#Получение комментариев
comments = train['comment_text']

In [ ]:
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# Токенизация

In [ ]:
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
#Обучение токенайзера
tokenizer.fit_on_texts(comments)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(comments)

In [ ]:
x_train = pad_sequences(train_sequences, maxlen=max_comment_len)

# Оптимизация гиперпараметров

In [ ]:
def build_model(hp):
    activation_choice = hp.Choice('activation', values=['sigmoid', 'tanh', 'relu', 'elu', 'selu'])
    optimizer_choice = hp.Choice('optimizer', values=['SGD', 'adam', 'adagrad', 'adadelta', 'rmsprop'])
    layers_conv_num = hp.Int('layers_num', 1, 3)
    neurons_num = hp.Int('neurons_num', min_value=12, max_value=256, step=32)
    filter_num = hp.Int('filter_num', min_value=32, max_value=512, step=32)
    kernel_size = hp.Int('kernel_size', min_value=1, max_value=5, step=1)
    embedding_num = hp.Int('embedding_num', min_value=4, max_value=128, step=16)
    embeddings_regularizer = keras.regularizers.l2(
        hp.Choice('embeddings_regularizer', values=[1e-2, 1e-3, 1e-4], default=1e-3))
    kernel_regularizer = keras.regularizers.l2(hp.Choice('kernel_regularizer', values=[1e-2, 1e-3, 1e-4], default=1e-3))

    model = Sequential()
    model.add(Embedding(input_dim=num_words, output_dim=embedding_num, input_length=max_comment_len,
                        embeddings_regularizer=embeddings_regularizer))
    for i in range(layers_conv_num):
        model.add(Conv1D(filters=filter_num, kernel_size=kernel_size, activation=activation_choice, kernel_regularizer=kernel_regularizer))
        model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=neurons_num, activation=activation_choice, kernel_regularizer=kernel_regularizer))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=optimizer_choice, loss='binary_crossentropy', metrics=[keras.metrics.AUC(name="auc"), "accuracy"])

    return model

In [ ]:
tuner = BayesianOptimization(
    build_model,
    objective=Objective('val_auc', 'max'),
    max_trials=50,
    directory='cnn_toxic_dir'
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
activation (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'tanh', 'relu', 'elu', 'selu'], 'ordered': False}
optimizer (Choice)
{'default': 'SGD', 'conditions': [], 'values': ['SGD', 'adam', 'adagrad', 'adadelta', 'rmsprop'], 'ordered': False}
layers_num (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
neurons_num (Int)
{'default': None, 'conditions': [], 'min_value': 12, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filter_num (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
embedding_num (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
embeddings_regularizer (Choice)
{'default': 0.001, 'conditions': [], 'values

In [15]:
tuner.search(x_train, y_train, batch_size=1024, epochs=5, validation_split=0.2)

Trial 50 Complete [00h 00m 42s]
val_auc: 0.7113415598869324

Best val_auc So Far: 0.9687618613243103
Total elapsed time: 00h 23m 52s


In [16]:
results_summary = tuner.results_summary(num_trials=50)

Results summary
Results in cnn_toxic_dir/untitled_project
Showing 50 best trials
Objective(name="val_auc", direction="max")

Trial 45 summary
Hyperparameters:
activation: elu
optimizer: rmsprop
layers_num: 1
neurons_num: 108
filter_num: 32
kernel_size: 5
embedding_num: 36
embeddings_regularizer: 0.0001
kernel_regularizer: 0.0001
Score: 0.9687618613243103

Trial 36 summary
Hyperparameters:
activation: selu
optimizer: rmsprop
layers_num: 1
neurons_num: 12
filter_num: 32
kernel_size: 5
embedding_num: 20
embeddings_regularizer: 0.0001
kernel_regularizer: 0.0001
Score: 0.9682829976081848

Trial 31 summary
Hyperparameters:
activation: selu
optimizer: rmsprop
layers_num: 1
neurons_num: 12
filter_num: 32
kernel_size: 5
embedding_num: 68
embeddings_regularizer: 0.0001
kernel_regularizer: 0.0001
Score: 0.9670343995094299

Trial 35 summary
Hyperparameters:
activation: selu
optimizer: rmsprop
layers_num: 1
neurons_num: 12
filter_num: 32
kernel_size: 5
embedding_num: 52
embeddings_regularizer: 0.00

# Загрузка тестовых данных

In [17]:
test_comments = pd.read_csv('drive/MyDrive/toxicCommentsDataSet/toxicCommentsTest.csv')

In [18]:
test_labels = pd.read_csv('drive/MyDrive/toxicCommentsDataSet/toxicCommentsTestLabels.csv')

In [19]:
test_full = pd.merge(test_comments, test_labels, on='id')

In [20]:
test = test_full[test_full['toxic']!=-1]

# Токенизация

In [21]:
test_sequences = tokenizer.texts_to_sequences(test['comment_text'])

In [22]:
y_test = test[['toxic',	'severe_toxic',	'obscene', 'threat',	'insult',	'identity_hate']]

In [23]:
x_test = pad_sequences(test_sequences, maxlen=max_comment_len)

# Проверка лучшей модели

In [24]:
model = tuner.get_best_models(num_models=1)

In [25]:
model[0].evaluate(x_test, y_test)

2000/2000 [==============================] - 6s 3ms/step - loss: 0.1116 - auc: 0.9609 - accuracy: 0.9386


[0.11155866086483002, 0.9609448909759521, 0.9385726451873779]

# Сохранение модели

In [26]:
model[0].save('drive/MyDrive/toxicCommentsDataSet/toxicCommentCNNModel.h5')